This Colab notebook demonstrates how to run our workflow using CMA-ES optimization on an example molecular dynamics (MD) dataset. The training process is skipped here, as it is identical to the one shown in the Bayesian Optimization example.

# Download and import

In [ ]:
#@title Download Data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.7 MB/s eta 0:00:00


In [ ]:
import os
os.chdir('/content/drive/My Drive/BorosilicateGlassesPotentialOptimization')

In [ ]:
import utils
from utils.data_process import data_process_NNW, load_data, normalize_NNW_data, data_process_bayes, bayes_search_space,store_optimization_results
from utils.model import create_model, train_model, VerboseCallback

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import tqdm
import copy
import csv
import re
import pytz
from datetime import datetime

from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

# Optimization

The example code runs CMA-ES using a trained MLP model, with weights provided in the model folder under the filename CMA_Model_weight.pth.

## Data Process

In [ ]:
data_path = '/content/drive/My Drive/BorosilicateGlassesPotentialOptimization/data/MD_Dataset.xlsx'
new_weight_path = '/content/drive/My Drive/BorosilicateGlassesPotentialOptimization/model/CMA_Model_weight.pth' #Path to the trained model weight
new_normalized_value_path = '/content/drive/My Drive/BorosilicateGlassesPotentialOptimization/model/CMA_Normalized_value.pth' #path to the normalized value used for trained MLP

In [ ]:
raw_data_df = load_data(data_path)

In [ ]:
normalized_values = torch.load(new_normalized_value_path, weights_only=True)

normalized_values

{'x_min': tensor([[3.8178e+06, 1.1780e-01, 5.6501e+02, 9.4950e+03, 2.9750e-01]]),
 'x_max': tensor([[5.7266e+06, 1.4260e-01, 9.2824e+02, 1.2846e+04, 4.0250e-01]]),
 'y_min': tensor(1.8391),
 'y_max': tensor(2.7013)}

In [ ]:
#Get the Original parameter value and experimental target
original_parameter, y_target_tensor = data_process_bayes(raw_data_df, no_0B = True)

In [ ]:
y_target_tensor[:, 0] = (y_target_tensor[:, 0] - normalized_values['y_min']) / (normalized_values['y_max'] - normalized_values['y_min'])

## CMA-ES

In [ ]:
#Set the search bond for CMA-ES
x0 = []
lower_bounds = []
upper_bounds = []
for ori_val, (low, high) in zip(original_parameter, percentage_ranges):
    low = ori_val * (1 + low/100)
    high = ori_val * (1 + high/100)
    mid = (low + high)/2
    x0.append(mid)
    lower_bounds.append(low)
    upper_bounds.append(high)

x0 = np.array(x0, dtype=float)

In [ ]:
lower_bounds

[np.float64(3817764.9208),
 np.float64(0.11779999999999999),
 np.float64(605.3722417500001),
 np.float64(9494.9499628),
 np.float64(0.2975)]

In [ ]:
upper_bounds

[np.float64(5726647.3812),
 np.float64(0.14259999999999998),
 np.float64(847.5211384500001),
 np.float64(12846.108773199998),
 np.float64(0.40249999999999997)]

In [ ]:
# Load the trained MLP model
state_dict = torch.load(new_weight_path)

In [ ]:
model = create_model()
model.load_state_dict(state_dict)
model.eval()

Sequential(
  (0): Linear(in_features=6, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=16, bias=True)
  (7): ReLU()
  (8): Linear(in_features=16, out_features=2, bias=True)
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Move model and tensors to GPU
model = model.to(device)
y_target_tensor = y_target_tensor.to(device)

cpu


In [ ]:
#Defind the objective function
def objective(params):
    #weighted loss: 1 * density loss + 2 * weighted B4 loss
    #weighted B4 loss: B4 MSE loss * Boron_num/0.75
    weight_col1 = 2.0
    boron_tensor = torch.tensor(boron_num, dtype=torch.float32).unsqueeze(1)
    boron_weight = (boron_tensor / 0.75).to(device).squeeze(1)
    #boron_weight = torch.tensor([0.5, 0.5, 0.5, 0.5, 0.75, 0.75, 1], dtype=torch.float32).to(device)

    parameters = torch.tensor(params, dtype=torch.float32)
    norm_params = (parameters - normalized_values['x_min']) / (normalized_values['x_max'] - normalized_values['x_min'])  # Example normalization
    mse_values = []

    norm_params_batch = norm_params.repeat(len(boron_num), 1)
    input_tensor_batch = torch.cat((norm_params_batch, boron_tensor), dim=1)
    input_tensor_batch = input_tensor_batch.to(device)
    outputs = model(input_tensor_batch)

    weight_vector = torch.tensor([1.0, weight_col1], device=device)
    diff = (outputs - y_target_tensor)**2

    diff[:, 1] *= boron_weight
    weighted_diff = diff * weight_vector   # broadcast multiplies col-wise

    mse_per_row = weighted_diff.mean(dim=1)  # shape [7]
    total_mse = mse_per_row.sum().item()

    return total_mse

In [ ]:
!pip install cma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 4.3 MB/s eta 0:00:00


In [ ]:
import cma
best_value_overall = []
best_solution_overall = []

range_values = np.array(upper_bounds) - np.array(lower_bounds)
sigma_vec = np.exp(np.random.uniform(-1, 1, size=len(x0))) * range_values * 1 #0.5
sigma = np.mean(range_values) * np.random.uniform(0.5, 2.0)
print(sigma_vec)

options = {
    'bounds': [lower_bounds, upper_bounds],
    'CMA_stds': sigma_vec,     # per-parameter initial std
    'maxiter': 5000,
    'popsize': max(1000, 4 + int(3 * np.log(len(x0))) * 10),
    'tolfacupx': 1e9,
    'tolfun': 1e-10,
    'tolfunhist': 1e-10,
    'tolstagnation':300,
    'verb_disp': 100         # print progress every 100 iterations
    # other settings...
}

for run_id in range(15):
    x0_random = lower_bounds + np.random.uniform(0, 1, size=len(x0)) ** 2 * range_values
    res = cma.fmin(objective, x0_random, sigma, options=options) #, restarts=10
    best_value = res[1]
    best_solution = res[0]

    if best_value not in best_value_overall:
        best_value_overall.append(best_value)
        best_solution_overall.append(best_solution)


print("Best across restarts:", best_value_overall)

[1.95420785e+06 4.43667120e-02 2.55791710e+02 4.37737708e+03
 2.68120650e-01]
(500_w,1000)-aCMA-ES (mu_w=254.6,w_1=1%) in dimension 5 (seed=806254, Tue Apr 15 21:31:39 2025)


/usr/local/lib/python3.11/dist-packages/cma/evolution_strategy.py:1244: UserWarning: Sampling standard deviation i=0 at iteration 0 change by 7.510698490250373e-07 to stds[0]=636294.1534666665
  warnings.warn("Sampling standard deviation i={0} at iteration {1}"
/usr/local/lib/python3.11/dist-packages/cma/evolution_strategy.py:1244: UserWarning: Sampling standard deviation i=1 at iteration 0 change by 4.2979562195216106e-07 to stds[1]=0.008266666666666663
  warnings.warn("Sampling standard deviation i={0} at iteration {1}"
/usr/local/lib/python3.11/dist-packages/cma/evolution_strategy.py:1244: UserWarning: Sampling standard deviation i=2 at iteration 0 change by 7.27879079323011e-07 to stds[2]=80.7162989
  warnings.warn("Sampling standard deviation i={0} at iteration {1}"
/usr/local/lib/python3.11/dist-packages/cma/evolution_strategy.py:1244: UserWarning: Sampling standard deviation i=3 at iteration 0 change by 5.886266415518012e-07 to stds[3]=1117.0529367999993
  warnings.warn("Samplin

Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1   1000 1.987414807081223e-02 1.0e+00 6.57e+05  8e-03  6e+05 0:00.6
    2   2000 1.263514254242182e-02 1.3e+00 8.25e+05  6e-03  6e+05 0:01.4
    3   3000 1.025767438113689e-02 2.3e+00 9.81e+05  3e-03  6e+05 0:02.3
    8   8000 8.572287857532501e-03 3.7e+01 1.95e+06  2e-03  5e+05 0:05.6
   12  12000 7.988950237631798e-03 3.7e+01 3.35e+06  4e-04  2e+05 0:09.6
   20  20000 7.760493084788322e-03 5.5e+01 1.42e+07  2e-05  3e+04 0:14.7
   27  27000 7.757451850920916e-03 6.5e+01 2.46e+07  9e-07  2e+03 0:21.1
   35  35000 7.757427170872688e-03 1.6e+02 3.65e+07  2e-07  4e+02 0:28.1
   46  46000 7.757430430501699e-03 5.5e+02 4.84e+07  6e-08  2e+02 0:36.5
   55  55000 7.757430430501699e-03 2.5e+03 6.12e+07  4e-09  2e+02 0:44.7
termination on tolfunhist=1e-10 (Tue Apr 15 21:32:33 2025)
final/bestever f-value = 7.757430e-03 7.757427e-03 after 55001/34732 evaluations
incumbent solution: [np.float64(3817764.9215155197), np.fl

In [ ]:
best_value_overall

[0.007757427170872688,
 0.00308796763420105,
 0.0030427773017436266,
 0.002940921811386943,
 0.003153787227347493,
 0.007202661130577326,
 0.00349793815985322,
 0.005290053319185972,
 0.007595100440084934,
 0.002935469849035144,
 0.0031537869945168495,
 0.020981814712285995]